In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import sys
import os

In [2]:
# Setting path to load util functions
from pathlib import Path
parent_dir = Path.cwd().parents[1]
sys.path.append(os.path.abspath(parent_dir))

In [3]:
from utils.TimeSeriesFeatureCreation import TimeSeriesFeatureCreation

In [4]:
selected_features = pd.read_csv('../../utils/selected_features/boruta_features_baseline.csv')
selected_features = selected_features[selected_features['Importance'] == 1]['Feature'].values

In [5]:
cols_to_append = ['customer_ID', 'end_of_month']
cols_to_append = np.append(cols_to_append, selected_features)
cols_to_append

array(['customer_ID', 'end_of_month', 'D_63_CL', 'B_14', 'D_60', 'S_11',
       'D_65', 'B_18', 'R_6', 'S_13', 'D_71', 'S_15', 'B_23', 'D_58',
       'P_4', 'R_10', 'R_11', 'R_12', 'B_28', 'R_16', 'S_26', 'D_102',
       'B_37', 'B_40', 'D_133', 'D_75', 'R_5', 'B_24', 'B_12', 'D_63_CO',
       'D_63_CR', 'S_8', 'D_126_1.0', 'D_39', 'B_1', 'R_1', 'B_4', 'B_5',
       'R_2', 'D_47', 'D_144', 'D_51', 'B_11', 'S_5', 'R_3', 'B_9',
       'B_10', 'R_4', 'B_7'], dtype=object)

In [6]:
df = pd.read_parquet('../../data/non-null/df_13_months_unprocessed_non_null.parquet').drop(columns=['S_2'])

In [7]:
train_ids = pd.read_parquet('../../data/3d_array/train_targets.parquet')['customer_ID'].values
val_ids = pd.read_parquet('../../data/3d_array/val_targets.parquet')['customer_ID'].values
test_ids = pd.read_parquet('../../data/3d_array/test_targets.parquet')['customer_ID'].values

In [8]:
train_df = df[df['customer_ID'].isin(train_ids)]
val_df = df[df['customer_ID'].isin(val_ids)]
test_df = df[df['customer_ID'].isin(test_ids)]

In [9]:
df = None

In [10]:
train_df.shape, val_df.shape, test_df.shape

((3486098, 82), (387137, 82), (1658216, 82))

In [11]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

def one_hot_encode_categories(df, categorical_columns=None, drop_original=True, handle_unknown='error'):
    """
    One-hot encode categorical columns in a DataFrame.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input DataFrame containing categorical columns
    categorical_columns : list or None
        List of categorical column names to encode. If None, automatically detects categorical columns.
    drop_original : bool
        Whether to drop the original categorical columns
    handle_unknown : str
        Strategy for handling unknown categories in new data: 'error', 'ignore' or 'infrequent_if_exist'
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with one-hot encoded columns
    OneHotEncoder
        Fitted encoder for future transformations
    """
    # Make a copy to avoid modifying the original
    result_df = df.copy()
    
    # Automatically detect categorical columns if not specified
    if categorical_columns is None:
        categorical_columns = result_df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    if not categorical_columns:
        print("No categorical columns found to encode.")
        return result_df, None
        
    # Initialize the encoder
    encoder = OneHotEncoder(sparse_output=False, handle_unknown=handle_unknown, drop='if_binary')
    
    # Fit and transform the categorical columns
    encoded_array = encoder.fit_transform(result_df[categorical_columns])
    
    # Get feature names
    feature_names = encoder.get_feature_names_out(categorical_columns)
    
    # Create a DataFrame with the encoded features
    encoded_df = pd.DataFrame(encoded_array, columns=feature_names, index=result_df.index)
    
    # Combine with the original DataFrame
    if drop_original:
        # Drop the original categorical columns
        result_df = result_df.drop(columns=categorical_columns)
    
    # Concatenate the encoded columns with the original DataFrame
    result_df = pd.concat([result_df, encoded_df], axis=1)
    
    print(f"One-hot encoded {len(categorical_columns)} categorical columns into {len(feature_names)} binary features.")
    
    return result_df, encoder

In [12]:
cat_cols = ['D_126', 'D_63']
train_df[cat_cols] = train_df[cat_cols].astype('category')
val_df[cat_cols] = val_df[cat_cols].astype('category')
test_df[cat_cols] = test_df[cat_cols].astype('category')

In [13]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='if_binary')
ohe.fit_transform(train_df[cat_cols])

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [14]:
ohe.get_feature_names_out(cat_cols)

array(['D_126_-1.0', 'D_126_0.0', 'D_126_1.0', 'D_126_nan', 'D_63_CL',
       'D_63_CO', 'D_63_CR', 'D_63_XL', 'D_63_XM', 'D_63_XZ'],
      dtype=object)

In [15]:
train_df, cat_encoder = one_hot_encode_categories(train_df, handle_unknown='ignore', categorical_columns=cat_cols)

One-hot encoded 2 categorical columns into 10 binary features.


In [16]:
val_df = pd.concat([val_df.drop(columns=cat_encoder.feature_names_in_),pd.DataFrame(cat_encoder.transform(val_df[cat_encoder.feature_names_in_]), columns=cat_encoder.get_feature_names_out(),index=val_df.index)], axis=1)
test_df = pd.concat([test_df.drop(columns=cat_encoder.feature_names_in_),pd.DataFrame(cat_encoder.transform(test_df[cat_encoder.feature_names_in_]), columns=cat_encoder.get_feature_names_out(),index=test_df.index)], axis=1)

In [17]:
train_df.shape, val_df.shape, test_df.shape

((3486098, 90), (387137, 90), (1658216, 90))

In [18]:
ts_creator = TimeSeriesFeatureCreation(create_2_diff=False, verbose = True, id_col='customer_ID', date_col = 'end_of_month',
                                       rolling_window_size=6, span = 6, num_lags=3)

In [19]:
val_df = ts_creator.transform(val_df[cols_to_append])

Aggregated DataFrame type: <class 'pandas.core.frame.DataFrame'>


Creating time series features for each feature:   0%|          | 0/47 [00:00<?, ?it/s]

Creating features for D_63_CL
	Creating lag features for D_63_CL
	Creating cumulative features for D_63_CL
	Creating expanding window features for D_63_CL
	Creating time differencing features for D_63_CL
	Creating rolling window features for D_63_CL
	Creating ewm features for D_63_CL with span 6


Creating time series features for each feature:   2%|▏         | 1/47 [00:06<05:18,  6.92s/it]

Creating features for B_14
	Creating lag features for B_14
	Creating cumulative features for B_14
	Creating expanding window features for B_14
	Creating time differencing features for B_14
	Creating rolling window features for B_14
	Creating ewm features for B_14 with span 6


...

Creating features for R_4
	Creating lag features for R_4
	Creating cumulative features for R_4
	Creating expanding window features for R_4
	Creating time differencing features for R_4
	Creating rolling window features for R_4
	Creating ewm features for R_4 with span 6


Creating time series features for each feature:  98%|█████████▊| 46/47 [05:07<00:07,  7.34s/it]

Creating features for B_7
	Creating lag features for B_7
	Creating cumulative features for B_7
	Creating expanding window features for B_7
	Creating time differencing features for B_7
	Creating rolling window features for B_7
	Creating ewm features for B_7 with span 6


Creating time series features for each feature: 100%|██████████| 47/47 [05:14<00:00,  6.68s/it]


Concatenating 282 feature DataFrames.


In [20]:
val_df.to_parquet('../../data/ts_features/val_13_months_boruta_selected_ts_features.parquet', index=False, engine='pyarrow', compression='zstd')

In [21]:
val_df.shape

(387137, 1036)

In [22]:
test_df = ts_creator.transform(test_df[cols_to_append])
test_df.to_parquet('../../data/ts_features/test_13_months_boruta_selected_ts_features.parquet', index=False, engine='pyarrow', compression='zstd')

Aggregated DataFrame type: <class 'pandas.core.frame.DataFrame'>


Creating time series features for each feature:   0%|          | 0/47 [00:00<?, ?it/s]

Creating features for D_63_CL
	Creating lag features for D_63_CL
	Creating cumulative features for D_63_CL
	Creating expanding window features for D_63_CL
	Creating time differencing features for D_63_CL
	Creating rolling window features for D_63_CL
	Creating ewm features for D_63_CL with span 6


Creating time series features for each feature:   2%|▏         | 1/47 [00:27<21:13, 27.70s/it]

Creating features for B_14
	Creating lag features for B_14
	Creating cumulative features for B_14
	Creating expanding window features for B_14
	Creating time differencing features for B_14
	Creating rolling window features for B_14
	Creating ewm features for B_14 with span 6


...

Creating features for R_4
	Creating lag features for R_4
	Creating cumulative features for R_4
	Creating expanding window features for R_4
	Creating time differencing features for R_4
	Creating rolling window features for R_4
	Creating ewm features for R_4 with span 6


Creating time series features for each feature:  98%|█████████▊| 46/47 [21:46<00:28, 28.81s/it]

Creating features for B_7
	Creating lag features for B_7
	Creating cumulative features for B_7
	Creating expanding window features for B_7
	Creating time differencing features for B_7
	Creating rolling window features for B_7
	Creating ewm features for B_7 with span 6


Creating time series features for each feature: 100%|██████████| 47/47 [22:15<00:00, 28.41s/it]


Concatenating 282 feature DataFrames.


In [23]:
train_df = ts_creator.transform(train_df[cols_to_append])
train_df.to_parquet('../../data/ts_features/train_13_months_boruta_selected_ts_features.parquet', index=False, engine='pyarrow', compression='zstd')

Aggregated DataFrame type: <class 'pandas.core.frame.DataFrame'>


Creating time series features for each feature:   0%|          | 0/47 [00:00<?, ?it/s]

Creating features for D_63_CL
	Creating lag features for D_63_CL
	Creating cumulative features for D_63_CL
	Creating expanding window features for D_63_CL
	Creating time differencing features for D_63_CL
	Creating rolling window features for D_63_CL
	Creating ewm features for D_63_CL with span 6


Creating time series features for each feature:   2%|▏         | 1/47 [01:00<46:26, 60.57s/it]

Creating features for B_14
	Creating lag features for B_14
	Creating cumulative features for B_14
	Creating expanding window features for B_14
	Creating time differencing features for B_14
	Creating rolling window features for B_14
	Creating ewm features for B_14 with span 6


...

Creating features for R_4
	Creating lag features for R_4
	Creating cumulative features for R_4
	Creating expanding window features for R_4
	Creating time differencing features for R_4
	Creating rolling window features for R_4
	Creating ewm features for R_4 with span 6


Creating time series features for each feature:  98%|█████████▊| 46/47 [46:11<01:00, 60.59s/it]

Creating features for B_7
	Creating lag features for B_7
	Creating cumulative features for B_7
	Creating expanding window features for B_7
	Creating time differencing features for B_7
	Creating rolling window features for B_7
	Creating ewm features for B_7 with span 6


Creating time series features for each feature: 100%|██████████| 47/47 [47:11<00:00, 60.25s/it]


Concatenating 282 feature DataFrames.


In [ ]:
train_df.shape